In [1]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import kurtosis
import requests
%matplotlib inline

In [2]:
df=pd.read_excel('C:\\Users\\nafissaad\Downloads\\Spreadsheets-20231017T124722Z-001\\Spreadsheets\\NEDL_Portfolio_Optimisation_Trading_Costs.xlsx',index_col='Date')
df=df[df.columns[:7]]
df.columns=df[0:1].values[0]
df=df[1:]
df

,JPM,WMT,MSFT,KO,PFE,CAT,SP 500
Date,,,,,,,
2011-11-14,24.705299,46.650124,21.537199,24.791821,13.043631,72.89994,103.301254
2011-11-15,24.819157,45.517323,21.683163,24.868612,13.096357,73.307755,103.811737
2011-11-16,23.885593,44.899456,21.139864,24.506557,12.898625,72.310883,102.164955
2011-11-17,23.141773,44.939053,20.710085,24.36393,12.832716,70.845764,100.5429
2011-11-18,23.24044,45.335125,20.515478,24.645535,12.872263,70.936417,100.435875
...,...,...,...,...,...,...,...
2021-11-08,169.059998,149.270004,336.98999,56.330002,48.330002,214.25,468.929993
2021-11-09,167.779999,149.789993,335.950012,56.490002,47.299999,209.809998,467.380005
2021-11-10,167.619995,148.440002,330.799988,56.720001,49.02,206.669998,463.619995


In [3]:
returns=df.pct_change().dropna()
returns

,JPM,WMT,MSFT,KO,PFE,CAT,SP 500
Date,,,,,,,
2011-11-15,0.004609,-0.024283,0.006777,0.003097,0.004042,0.005594,0.004942
2011-11-16,-0.037615,-0.013574,-0.025056,-0.014559,-0.015098,-0.013598,-0.015863
2011-11-17,-0.031141,0.000882,-0.020330,-0.005820,-0.005110,-0.020261,-0.015877
2011-11-18,0.004264,0.008814,-0.009397,0.011558,0.003082,0.001280,-0.001064
2011-11-21,-0.023187,-0.009960,-0.011857,-0.021368,-0.029186,-0.029916,-0.019020
...,...,...,...,...,...,...,...
2021-11-08,0.006010,-0.006390,0.002767,-0.008973,-0.005760,0.040655,0.000854
2021-11-09,-0.007571,0.003484,-0.003086,0.002840,-0.021312,-0.020723,-0.003305
2021-11-10,-0.000954,-0.009013,-0.015330,0.004072,0.036364,-0.014966,-0.008045


In [4]:
matrix=pd.DataFrame(columns=[returns.columns],index=['Return,historical','Beta','Expected returns'])
risk_free_rate=0.0157
trading_cost=0.01
Investing_horaizon_days=252#252 trading Days
matrix

,JPM,WMT,MSFT,KO,PFE,CAT,SP 500
"Return,historical",NaN,NaN,NaN,NaN,NaN,NaN,NaN
Beta,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Expected returns,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
np.polyfit(returns['SP 500'],returns['JPM'],1)[0]

1.2576308464675616

In [6]:
matrix.loc['Return,historical','SP 500'].values

array([nan], dtype=object)

In [7]:
for i in df.columns:
    matrix.loc['Return,historical',i]=(df[i][-1]/df[i][0])**(1/10)-1
    matrix.loc['Beta',i]=np.polyfit(returns['SP 500'],returns[i],1)[0]
    ex=(df['SP 500'][-1]/df['SP 500'][0])**(1/10)-1
    matrix.loc['Expected returns',i]=risk_free_rate+matrix.loc['Beta',i]*(ex-risk_free_rate)
matrix    

,JPM,WMT,MSFT,KO,PFE,CAT,SP 500
"Return,historical",0.210476,0.1222,0.316461,0.086072,0.143199,0.111215,0.16291
Beta,1.257631,0.549411,1.171371,0.671961,0.722679,1.167853,1.0
Expected returns,0.200836,0.096579,0.188138,0.11462,0.122086,0.18762,0.16291


In [8]:
weights=pd.DataFrame(columns=[returns.columns[:-1]],index=['Initial weights','New weights'])
weights.loc['Initial weights']=0.2
weights.loc['Initial weights','JPM']=0
weights.loc['New weights']=1/6#weights.loc['Initial weights'].values
weights

,JPM,WMT,MSFT,KO,PFE,CAT
Initial weights,0,0.2,0.2,0.2,0.2,0.2
New weights,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667


In [9]:
returns[returns.columns[:-1]].dot(weights.loc['New weights'].values)

Date
2011-11-15   -0.000027
2011-11-16   -0.019917
2011-11-17    -0.01363
2011-11-18    0.003267
2011-11-21   -0.020912
                ...   
2021-11-08    0.004718
2021-11-09   -0.007728
2021-11-10    0.000029
2021-11-11    0.006381
2021-11-12   -0.000711
Length: 2516, dtype: object

In [10]:
Gross_return=weights.loc['Initial weights'].dot(matrix.loc['Expected returns'][:-1].values)**(Investing_horaizon_days/252)
Trading_volume=sum(abs(weights.loc['New weights'].values-weights.loc['Initial weights'].values))
Fees_paid=Trading_volume*trading_cost
net_return=(1+Gross_return)*(1-Fees_paid)-1#Net return=(1+Gross return)(1-Fees)-1
Risk=returns[returns.columns[:-1]].dot(weights.loc['New weights'].values).std()*np.sqrt(252)
Theta=3#Risk aversion Theta 0 to 5(higher theta indiacte lower risk tolerance )
Utility=net_return-Theta/2*Risk**2
Utility

0.09796235618394339

In [ ]:
#When trading cost is high we are less likely to trade and we would like to invest our money in consistant assets
#use solver to optimize utility subject to weights.